In [33]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

#### Download Data from datasets

In [34]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [35]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X [N C H W]', X.shape)
    print('Shape of y', y.shape, y.dtype)
    break

Shape of X [N C H W] torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]) torch.int64


#### Create Models

In [48]:
# Get cpu or gpu device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [49]:
# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### Optimizing the model

In [54]:
# Loss Function for classification problem
# Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [51]:
# Define single training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')
    

In [52]:
# Define test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [55]:
# Training Process
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n----------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done')

Epoch 1
----------------------
loss: 1.151295  [    0/60000]
loss: 1.052111  [ 6400/60000]
loss: 0.799238  [12800/60000]
loss: 0.930411  [19200/60000]
loss: 0.775528  [25600/60000]
loss: 0.746376  [32000/60000]
loss: 0.790883  [38400/60000]
loss: 0.735745  [44800/60000]
loss: 0.737109  [51200/60000]
loss: 0.689123  [57600/60000]
Test Error: 
 Accuracy: 75.5%, Avg loss: 0.693629 

Epoch 2
----------------------
loss: 0.642648  [    0/60000]
loss: 0.736390  [ 6400/60000]
loss: 0.492765  [12800/60000]
loss: 0.737475  [19200/60000]
loss: 0.632702  [25600/60000]
loss: 0.605512  [32000/60000]
loss: 0.618295  [38400/60000]
loss: 0.657723  [44800/60000]
loss: 0.657754  [51200/60000]
loss: 0.573974  [57600/60000]
Test Error: 
 Accuracy: 79.5%, Avg loss: 0.588433 

Epoch 3
----------------------
loss: 0.497717  [    0/60000]
loss: 0.617780  [ 6400/60000]
loss: 0.400755  [12800/60000]
loss: 0.652757  [19200/60000]
loss: 0.579993  [25600/60000]
loss: 0.554830  [32000/60000]
loss: 0.534948  [38400/

#### Saving Models

In [56]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model Sate to model.pth")

Saved Pytorch Model Sate to model.pth


#### Loading Models

In [57]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [84]:
# Make predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]  # argmax返回最大值的索引

print(f'Predicted: "{predicted}",  Acutal: "{actual}"')

Predicted: "Ankle boot",  Acutal: "Ankle boot"
